# Unificación de archivos finales de Divorcios y VIF para unirlo a un DF


In [155]:
import pandas as pd
import numpy as np

In [156]:
def moda_filtrada(s, filtro=None):
    # Convertir a numérico (evita errores como comparar str con int)
    s = pd.to_numeric(s, errors='coerce')
    
    if filtro is not None:
        s = s[s.apply(filtro)]
    
    if s.empty:
        return np.nan
    
    moda = s.mode()
    return moda.iloc[0] if not moda.empty else np.nan

In [ ]:
def obtener_ocupacion_dedica(grupo, col_trabaja, col_ocup, col_dedica):
    try:
        trabaja = grupo[col_trabaja].mode().iloc[0]
    except IndexError:
        return pd.Series([np.nan, np.nan], index=['OCUPACION', 'DEDICA'])

    if trabaja == 1:
        ocup_filtrada = grupo[col_ocup][grupo[col_ocup] != 9999]
        valor_ocup = ocup_filtrada.mode().iloc[0] if not ocup_filtrada.mode().empty else np.nan
        return pd.Series([valor_ocup, np.nan], index=['OCUPACION', 'DEDICA'])
    
    elif trabaja == 2:
        dedica_filtrada = grupo[col_dedica][grupo[col_dedica] != 9]
        valor_dedica = dedica_filtrada.mode().iloc[0] if not dedica_filtrada.mode().empty else np.nan
        return pd.Series([np.nan, valor_dedica], index=['OCUPACION', 'DEDICA'])

    else:
        return pd.Series([np.nan, np.nan], index=['OCUPACION', 'DEDICA'])

### Cargar y procesar datos de Divorcios


In [158]:
div = pd.read_csv("../CSV/UnidoPorAñoCSV/divorcio_all_years.csv", low_memory=False)
div.columns = div.columns.str.strip().str.upper()

div['DEPARTAMENTO'] = div['DEPOCU']
div['ANIO'] = div['AÑOREG']

div_agg = div.groupby(['DEPARTAMENTO','ANIO']).agg(
    edad_hombre=('EDADHOM', lambda s: s.replace(999, np.nan).mean()),
    edad_mujer=('EDADMUJ', lambda s: s.replace(999, np.nan).mean()),
    escolaridad_hombre=('ESCHOM', lambda s: moda_filtrada(s, lambda x: x != 9)), 
    escolaridad_mujer=('ESCMUJ', lambda s: moda_filtrada(s, lambda x: x != 9)),
    ocupacion_hombre=('CIUOHOM', lambda s: moda_filtrada(s, lambda x: x <= 96)), 
    ocupacion_mujer=('CIUOMUJ', lambda s: moda_filtrada(s, lambda x: x <= 96)),

    total_divorcios=('DEPOCU','count')
).reset_index()


### Cargar y procesar datos de VIF


In [159]:
vif = pd.read_csv("../CSV/UnidoPorAñoCSV/vif_all_years.csv", low_memory=False)
vif.columns = vif.columns.str.strip().str.upper()

vif['HEC_DEPTOMCPIO'] = vif['HEC_DEPTOMCPIO'].astype(str).str.replace(',', '').str.strip()

vif['DEPARTAMENTO'] = vif['HEC_DEPTOMCPIO'].str[:-2].astype(float).astype('Int64')

vif['ANIO'] = vif['HEC_ANO']

vif_agg = vif.groupby(['DEPARTAMENTO', 'ANIO']).agg(
    edad_victima=('VIC_EDAD', lambda s: s.replace(999, np.nan).mean()),
    edad_agresor=('AGR_EDAD', lambda s: s.replace(999, np.nan).mean()),

    escolaridad_victima=('VIC_ESCOLARIDAD', lambda s: moda_filtrada(s, lambda x: x != 99)),
    escolaridad_agresor=('AGR_ESCOLARIDAD', lambda s: moda_filtrada(s, lambda x: x != 99)),

    tipo_medida_mas_frec=('TIPO_MEDIDA', lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan),
    inst_denuncia=('INST_DENUN_HECHO', lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan),
    total_vif=('HEC_ANO', 'count')
    
).reset_index()


In [160]:
cols_to_numeric = ['VIC_TRABAJA', 'VIC_OCUP', 'VIC_DEDICA', 'AGR_TRABAJA', 'AGR_OCUP', 'AGR_DEDICA']
vif[cols_to_numeric] = vif[cols_to_numeric].apply(pd.to_numeric, errors='coerce')

# Agrupar por departamento y año
vif_grouped = vif.groupby(['DEPARTAMENTO', 'ANIO'])

# Obtener VICTIMA_TRABAJA y AGR_TRABAJA como moda
trabaja_modas = vif_grouped.agg({
    'VIC_TRABAJA': lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan,
    'AGR_TRABAJA': lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan
}).reset_index()

# Obtener ocupación/dedica condicionalmente
victima_cond = vif_grouped.apply(lambda g: obtener_ocupacion_dedica(g, 'VIC_TRABAJA', 'VIC_OCUP', 'VIC_DEDICA')).reset_index()
agresor_cond = vif_grouped.apply(lambda g: obtener_ocupacion_dedica(g, 'AGR_TRABAJA', 'AGR_OCUP', 'AGR_DEDICA')).reset_index()

# Renombrar columnas para evitar conflictos
victima_cond.columns = ['DEPARTAMENTO', 'ANIO', 'OCUPACION_VICTIMA', 'DEDICA_VICTIMA']
agresor_cond.columns = ['DEPARTAMENTO', 'ANIO', 'OCUPACION_AGRESOR', 'DEDICA_AGRESOR']

# Combinar todo con vif_agg
vif_agg = vif_agg.merge(trabaja_modas, on=['DEPARTAMENTO', 'ANIO'], how='left')
vif_agg = vif_agg.merge(victima_cond, on=['DEPARTAMENTO', 'ANIO'], how='left')
vif_agg = vif_agg.merge(agresor_cond, on=['DEPARTAMENTO', 'ANIO'], how='left')

C:\Users\ncast\AppData\Local\Temp\ipykernel_26088\4264002683.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  victima_cond = vif_grouped.apply(lambda g: obtener_ocupacion_dedica(g, 'VIC_TRABAJA', 'VIC_OCUP', 'VIC_DEDICA')).reset_index()
C:\Users\ncast\AppData\Local\Temp\ipykernel_26088\4264002683.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agresor_cond = vif_grouped.apply(lambda g: obtener_ocup

#### Unir datasets y limpiar


In [161]:
# Asegurar consistencia de tipos en claves para merge
div_agg['ANIO'] = pd.to_numeric(div_agg['ANIO'], errors='coerce').astype('Int64')
vif_agg['ANIO'] = pd.to_numeric(vif_agg['ANIO'], errors='coerce').astype('Int64')

df = pd.merge(div_agg, vif_agg, on=['DEPARTAMENTO', 'ANIO'], how='outer')
df.columns = df.columns.str.strip().str.upper()

# Rellenar valores nulos
df['TOTAL_DIVORCIOS'] = df['TOTAL_DIVORCIOS'].fillna(0).astype(int)
df['TOTAL_VIF'] = df['TOTAL_VIF'].fillna(0).astype(int)

# Rellenar proporciones nulas con 0
proportion_cols = [col for col in df.columns if col.startswith('PCT_')]
df[proportion_cols] = df[proportion_cols].fillna(0)

# Filtrar filas útiles
df = df[(df['TOTAL_DIVORCIOS'] > 0) | (df['TOTAL_VIF'] > 0)].reset_index(drop=True)

# Filtrar años relevantes
df = df[(df['ANIO'] >= 2009) & (df['ANIO'] <= 2018)]

print(" Dataset combinado y limpio listo para análisis.")


 Dataset combinado y limpio listo para análisis.


### Guardar dataset final


In [162]:
df.to_csv("../CSV/FinalCSV/dataset_unificado_limpio.csv", index=False, encoding='utf-8-sig')
print(" Dataset final limpio guardado como dataset_unificado_limpio.csv")

 Dataset final limpio guardado como dataset_unificado_limpio.csv
